<a href="https://colab.research.google.com/github/Mikolajkunka/Missp/blob/main/pulp/Whiskas_problem_2_metody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pulp # instalacja biblioteki pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp


     |████████████████████████████████| 40.6MB 98kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5 libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev default-libmysqlclient-dev
The following NEW packages will be installed:
  coinor-cbc coinor-clp coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5 glpk-utils libamd2 libcolamd2
  libglpk40 libsuitesparseconfig5
0 upgraded, 12 newly installed, 0 to remove and 14 not upgraded.
Need to get 3,439 kB of archives.
After this operation, 9,832 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libamd2 amd64 1:5.1.2-2 [19.5 kB]
Get:3 h

In [ ]:
### kod sprawdza czy instalacja się udała
import pulp

print(pulp.__version__) #sprawdzenie wersji 

2.3.1


In [3]:
pulp.pulpTestAll() # wykonanie testu biblioteki pulp

NameError: ignored

# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:



In [ ]:
Naszym zadaniem było rozwiązanie pewnego problemu:
https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html

Celem tej optymalizacji jest jak największy zysk przy produkcji jednej puszki 
karmy dla kotów. Musimy jednak pamiętać o tym, aby spełnić pewne założenia dotyczące 
na przykład zawartości makroskładników w karmie oraz proporcji poszczególnych składników,
które muszą zawierać się w przedziale od 0 do 100%. Przy pomocy poniższego kodu dobrana
zostanie odpowiednia proprcja składników oraz cena zapewniająca maksymalny zysk dla producenta.
Przetestowane zostały dwa modele uproszczony oraz rozwinięty, który zawiera więcej możliwych
do użycia składników.

In [ ]:

from pulp import * # import biblioteki pulp

#zmienna prob to zmienna do której będziemy dodawać ograniczenia
#LpMinimize ma za zadanie minimalizacje kosztu produkcji
prob = LpProblem("The Whiskas Problem",LpMinimize)
LpVariable("example", None, 100)

# The 2 variables Beef and Chicken are created with a lower limit of zero
#zmienne x1,x2,x3 oznaczają poszczególne składniki(porcentowa zawartość)
x1=LpVariable("ChickenPercent",0,None,LpInteger)
x2=LpVariable("BeefPercent",0)
x3=LpVariable("MuttonPercent",0)

# The objective function is added to 'prob' first
prob += 0.013*x1 + 0.008*x2 + 0.004*x3, "Total Cost of Ingredients per can"

# The five constraints are entered
prob += x1 + x2 + x3 == 100, "PercentagesSum"
#tak jak zostało opisane to w naszym problemie, musimy dodać pewne wymagania
# takie jak minimalna zawartość poszczególnych mikroskładników oraz suma procetowa
prob += 0.100*x1 + 0.200*x2 + 0.0*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 + 0.11*x3 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 + 0.003*x3 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.0075*x3 <= 0.4, "SaltRequirement"
prob += 0.003*x1 + 0.026*x2 + 0.026*x3 >= 0.8, "Witamina B" 

prob.writeLP("WhiskasModel.lp") # nazwa Naszego problemu
prob.solve() # funkcja wywołująca rozwiązanie
print("Status:", LpStatus[prob.status])  # informacja o statusie problemu i możliwości jego rozwiązania

for v in prob.variables():
    print(v.name, "=", v.varValue)  #wypisanie każdego użytego składnika wraz z zawartością %

udzial = ""

for v in prob.variables():
    udzial += v.name + " = " + str(v.varValue)+ "%, "  #możliwość wypisania każdego produktu w jednej linii

udzial = udzial[:-2]
print("Cena puszki to: ", value(prob.objective), "$ i będzie się składać z:", udzial,".")


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
BeefPercent = 65.2
ChickenPercent = 34.0
MuttonPercent = 0.8
Cena puszki to:  0.9668 $ i będzie się składać z: BeefPercent = 65.2%, ChickenPercent = 34.0%, MuttonPercent = 0.8% .


In [ ]:
W drugim wariancie mamy do czynienia z wersją rozbudowaną, w której wszystkie
składniki odżywcze są zgrupowane w poszczególne słowniki, dzięki czemu mamy do nich
lepszy dostęp w przypadku modyfikacji. Daje to szczególne korzyści w przypadku, gdy
optymalizacji poddamy rozbudowany problem.

In [11]:
from pulp import *

#tabela zawierająca listę produktów
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001,
         'SARDINES': 0.015}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'SARDINES': 0.150}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'SARDINES': 0.008}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES': 0.000}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'SARDINES': 0.001}
# A dictionary of the Vit. B percent in each of the Ingredients is created
WitaminaBPercent = {'CHICKEN': 0.003, 
               'BEEF': 0.026, 
               'MUTTON': 0.026, 
               'RICE': 0.0, 
               'WHEAT': 0.0, 
               'GEL': 0.0,
               'SARDINES': 0.89}

prob = LpProblem("The Whiskas Problem", LpMinimize)

#utworzenie słownika dla zmiennych
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)

#ze słownika "costs" pobieramy cene produktu i mnożymy ją przed odpowiednią zmienną x
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

#ograniczenie procentowe sumy wszystkich składników
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"

#warunki poszczególnych wartości odżywczych tak jak w modelu uproszczonym
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([WitaminaBPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.8, "WitaminaBRequirement"
prob += ingredient_vars['SARDINES'] >=25, "MinSardines"
prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

udzial = ""

for v in prob.variables():
    udzial += v.name + " = " + str(v.varValue)+ "%, " 

udzial = udzial[:-2]
print("Cena puszki to: ", value(prob.objective), "$ i będzie się składać z:", udzial,".")


Status: Optimal
Ingr_BEEF = 58.0
Ingr_CHICKEN = 0.0
Ingr_GEL = 17.0
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_SARDINES = 25.0
Ingr_WHEAT = 0.0
Cena puszki to:  0.8560000000000001 $ i będzie się składać z: Ingr_BEEF = 58.0%, Ingr_CHICKEN = 0.0%, Ingr_GEL = 17.0%, Ingr_MUTTON = 0.0%, Ingr_RICE = 0.0%, Ingr_SARDINES = 25.0%, Ingr_WHEAT = 0.0% .


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
Po impelementacji wersji rozszerzonej początkowo wartość została zoptymalizowana w taki sposób,
że do produkcji kociej karmy zostały użyte tylko dwa skladniki "beef" 60% oraz "gel" 40%.
Aby urozmaicić skład pożywienia i zapewnić różnorodny smak dodano ograniczenie, co do 
minimalnej zawartości sardynek powyżej 25%. Efekt takiej operacji, czyli cena i zawartość 
procentowa widoczna jest powyżej.